## 1

In [1]:
from itertools import combinations

transacoes = [
    ['tree', 'cup', 'paper'],
    ['book', 'tomato', 'pen'],
    ['pen', 'book', 'tree'],
    ['tomato', 'pen', 'cup'],
    ['tree', 'paper', 'book'],
    ['pen', 'book']
]

def calcular_suporte(conjunto, transacoes):
    contador = 0
    for transacao in transacoes:
        if set(conjunto).issubset(transacao):
            contador += 1
    return contador / len(transacoes) * 100

itens_distintos = list(set([item for transacao in transacoes for item in transacao]))

conjuntos_suporte = {}
for tamanho in range(2, len(itens_distintos) + 1):
    for conjunto in combinations(itens_distintos, tamanho):
        suporte = calcular_suporte(list(conjunto), transacoes)
        if suporte >= 33:
            conjuntos_suporte[str(conjunto)] = suporte

for conjunto, suporte in conjuntos_suporte.items():
    print(f"{conjunto}: {suporte}%")


('book', 'tree'): 33.33333333333333%
('book', 'pen'): 50.0%
('tree', 'paper'): 33.33333333333333%
('tomato', 'pen'): 33.33333333333333%


## 2

In [1]:
from itertools import combinations
from collections import defaultdict

# Conjunto de transações
transacoes = [
    {1, 3, 5, 7, 8},
    {3, 4, 5, 7},
    {2, 3, 5, 6},
    {2, 4, 5, 8, 9},
    {2, 3, 5, 11},
    {1, 2, 3, 7, 9}
]

def calcular_suporte(conjunto, transacoes):
    contador = 0
    for transacao in transacoes:
        if conjunto.issubset(transacao):
            contador += 1
    return contador / len(transacoes)

def gerar_regras(transacoes, confianca_minima, numeros_alvo):
    regras = defaultdict(list)
    itens_distintos = set([item for transacao in transacoes for item in transacao])
    
    suportes_individuais = {item: calcular_suporte({item}, transacoes) for item in itens_distintos}
    
    conjuntos_dois_itens = list(combinations(itens_distintos, 2))
    
    for conjunto in conjuntos_dois_itens:
        if set(conjunto).issubset(numeros_alvo):
            suporte_conjunto = calcular_suporte(set(conjunto), transacoes)
            confianca = suporte_conjunto / suportes_individuais[conjunto[0]]
            
            if confianca >= confianca_minima:
                regras[conjunto[0]].append((conjunto[1], confianca))
    
    return regras

numeros_alvo = {2, 3, 5, 7}

confianca_minima = 0.73
regras_geradas = gerar_regras(transacoes, confianca_minima, numeros_alvo)

print("Regras com confiança >= 73%")
print("Regra:Confiança")
for item, regras in regras_geradas.items():
    for regra in regras:
        print(f"{item} -> {regra[0]} ({regra[1]*100:.2f}%)")


Regras com confiança >= 73%
Regra:Confiança
2 -> 3 (75.00%)
2 -> 5 (75.00%)
3 -> 5 (80.00%)


## 3

In [10]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

with open('supermercado.csv', 'r') as arquivo:
    linhas = arquivo.readlines()

transacoes = [linha.strip().split(',') for linha in linhas]

te = TransactionEncoder()
te_array = te.fit(transacoes).transform(transacoes)

df_codificado = pd.DataFrame(te_array, columns=te.columns_)

itemsets_frequentes = apriori(df_codificado, min_support=0.001, use_colnames=True)

regras = association_rules(itemsets_frequentes, metric='confidence', min_threshold=0.25)
regras_filtradas = regras[regras['consequents'].apply(lambda x: 'bottled beer' in x)]

for indice, linha in regras_filtradas.iterrows():
    antecedentes = ', '.join(list(linha['antecedents']))
    consequentes = ', '.join(list(linha['consequents']))
    print(f"Regra: {antecedentes} -> {consequentes}")
    print(f" - Suporte: {linha['support']:.4f}")
    print(f" - Confiança: {linha['confidence']:.4f}")
    print(f" - Lift: {linha['lift']:.4f}")
    print("------------------------------")


Regra: liquor -> bottled beer
 - Suporte: 0.0047
 - Confiança: 0.4220
 - Lift: 5.2406
------------------------------
Regra: red/blush wine -> bottled beer
 - Suporte: 0.0049
 - Confiança: 0.2540
 - Lift: 3.1538
------------------------------
Regra: tea -> bottled beer
 - Suporte: 0.0011
 - Confiança: 0.2895
 - Lift: 3.5947
------------------------------
Regra: bottled water, herbs -> bottled beer
 - Suporte: 0.0012
 - Confiança: 0.4000
 - Lift: 4.9672
------------------------------
Regra: bottled water, oil -> bottled beer
 - Suporte: 0.0012
 - Confiança: 0.2927
 - Lift: 3.6345
------------------------------
Regra: bottled water, sugar -> bottled beer
 - Suporte: 0.0012
 - Confiança: 0.2553
 - Lift: 3.1705
------------------------------
Regra: butter, onions -> bottled beer
 - Suporte: 0.0010
 - Confiança: 0.2500
 - Lift: 3.1045
------------------------------
Regra: red/blush wine, liquor -> bottled beer
 - Suporte: 0.0019
 - Confiança: 0.9048
 - Lift: 11.2353
-------------------------